In [1]:
import os
import sys
import glob
sys.path.append(os.path.dirname(os.getcwd()))

import torch
import numpy as np
from torchvision.transforms import transforms
from efficientnet_pytorch import EfficientNet

from core.dataset import ImageDataset

In [2]:
os.getcwd()

'/home/oem/repositories/BreastImplantRupture/notebooks'

In [3]:
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
model = EfficientNet.from_pretrained(
    "efficientnet-b3", num_classes=1, image_size=512
)
dd_model = torch.nn.DataParallel(model).to("cuda:3")
dd_model.load_state_dict(torch.load("../data/checkpoints/rupture_ratio_crop/best_weight.pt"))
#dd_model.load_state_dict(torch.load("../data/checkpoints/texture/best_weight.pt"))
_ = model.eval()

Loaded pretrained weights for efficientnet-b3


In [8]:
from torchsummary import summary

ModuleNotFoundError: No module named 'torchsummary'

In [7]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

10697769

In [11]:
from core.evaluation import forward, plot_confusion_matrix
from core.evaluation import save_gradcam_by_dataset
from matplotlib import pyplot as plt 
from core.dataset import ElasticDistDataset

def crop_numpy_array(np_array):
    """
    넘파이 배열의 상위 15%와 하위 10%를 잘라내어 새로운 넘파이 배열을 반환합니다.

    Args:
        np_array (numpy.ndarray): 처리할 넘파이 배열

    Returns:
        numpy.ndarray: 잘라낸 넘파이 배열
    """
    height = np_array.shape[0]

    # 상위 15%와 하위 10%의 높이를 계산합니다.
    top_crop = int(height * 0.1468)
    bottom_crop = int(height * 0.864)

    # 넘파이 배열을 잘라냅니다.
    cropped_np_array = np_array[top_crop:bottom_crop, :]

    return cropped_np_array


ALL_DATASET = "/home/oem/repositories/BreastImplantRupture/data/test_rupture"
for camera in os.listdir("/home/oem/repositories/BreastImplantRupture/data/test_rupture"):  
    
    print(camera)       
    subtest_dir = os.path.join(ALL_DATASET, camera)
    test_normal_paths = glob.glob(os.path.join(subtest_dir, "normal/*.jpg"))
    test_rupture_paths = glob.glob(os.path.join(subtest_dir, "rupture/*.jpg"))

    test_dataset = ElasticDistDataset(
        test_rupture_paths + test_normal_paths,
        transform=transform,
        label_hint="/rupture/",
        crop=crop_numpy_array,
        distortion=None,
        device="cuda:3",
    )
    y_trues, y_probs = forward(model, test_dataset, return_probs=True)
    y_preds = y_probs >= 0.5
    plot_confusion_matrix(y_trues, y_preds, label_names=["Normal", "Rupture"])
    
    save_dir = os.path.join("performance.ratio_crop", camera)
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(os.path.join(save_dir, "Performance.png"))
    plt.clf()
    
        
    save_gradcam_by_dataset(model, test_rupture_paths + test_normal_paths, test_dataset, crop=crop_numpy_array, save_dir=save_dir)

canon


  6%|█████████▎                                                                                                                                                          | 97/1700 [00:06<01:48, 14.73it/s]


KeyboardInterrupt: 

In [4]:

camera = "inhouse"
    
print(camera)       
subtest_dir = "/home/oem/repositories/BreastImplantRupture/data/testset"
test_normal_paths = glob.glob(os.path.join(subtest_dir, "normal/*.jpg"))
test_rupture_paths = glob.glob(os.path.join(subtest_dir, "rupture/*.jpg"))

test_dataset = ElasticDistDataset(
    test_rupture_paths + test_normal_paths,
    transform=transform,
    label_hint="/rupture/",
    crop=crop_numpy_array,
    distortion=None,
    device="cuda:3",
)
y_trues, y_probs = forward(model, test_dataset, return_probs=True)
y_preds = y_probs >= 0.5
plot_confusion_matrix(y_trues, y_preds, label_names=["Normal", "Rupture"])

save_dir = os.path.join("performance.ratio_crop", camera)
os.makedirs(save_dir, exist_ok=True)
plt.savefig(os.path.join(save_dir, "Performance.png"))
plt.clf()


save_gradcam_by_dataset(model, test_rupture_paths + test_normal_paths, test_dataset, crop=crop_numpy_array, save_dir=save_dir)

inhouse


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 509/509 [00:37<00:00, 13.63it/s]
0it [00:00, ?it/s]/home/oem/anaconda3/envs/breast/lib/python3.8/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
36it [00:09,  3.04it/s]/home/oem/anaconda3/envs/breast/lib/python3.8/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  wa

236it [01:29,  2.47it/s]/home/oem/anaconda3/envs/breast/lib/python3.8/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
238it [01:29,  2.43it/s]/home/oem/anaconda3/envs/breast/lib/python3.8/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
464it [02:31,  2.92it/s]/home/oem/anaconda

<Figure size 1000x800 with 0 Axes>

In [ ]:

camera = "inhouse"
    
print(camera)       
subtest_dir = "/home/oem/repositories/BreastImplantRupture/data/test_rupture/others"
test_normal_paths = glob.glob(os.path.join(subtest_dir, "normal/*.jpg"))
test_rupture_paths = glob.glob(os.path.join(subtest_dir, "rupture/*.jpg"))

test_dataset = ElasticDistDataset(
    test_rupture_paths + test_normal_paths,
    transform=transform,
    label_hint="/rupture/",
    crop=crop_numpy_array,
    distortion=None,
    device="cuda:3",
)
y_trues, y_probs = forward(model, test_dataset, return_probs=True)
y_preds = y_probs >= 0.5
plot_confusion_matrix(y_trues, y_preds, label_names=["Normal", "Rupture"])

save_dir = os.path.join("performance.ratio_crop", camera)
os.makedirs(save_dir, exist_ok=True)
plt.savefig(os.path.join(save_dir, "Performance.png"))
plt.clf()


save_gradcam_by_dataset(model, test_rupture_paths + test_normal_paths, test_dataset, crop=crop_numpy_array, save_dir=save_dir)